# Env

In [1]:
!conda env list

# conda environments:
#
deepface_env          *  /home/dzega/.conda/envs/deepface_env
env_full                 /home/dzega/.conda/envs/env_full
face_recognition_env     /home/dzega/.conda/envs/face_recognition_env
jupyterlab               /storage/modules/envs/jupyterlab
base                     /storage/modules/packages/anaconda



In [2]:
!conda list

# packages in environment at /home/dzega/.conda/envs/deepface_env:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main    defaults
_openmp_mutex             5.1                       1_gnu    defaults
asttokens                 3.0.0                    pypi_0    pypi
ca-certificates           2024.12.31           h06a4308_0    defaults
certifi                   2025.1.31                pypi_0    pypi
charset-normalizer        3.4.1                    pypi_0    pypi
comm                      0.2.2                    pypi_0    pypi
debugpy                   1.8.12                   pypi_0    pypi
decorator                 5.1.1                    pypi_0    pypi
exceptiongroup            1.2.2                    pypi_0    pypi
executing                 2.2.0                    pypi_0    pypi
facenet-pytorch           2.6.0                    pypi_0    pypi
filelock                  3.13.1                   pypi_0   

# Imports

In [1]:
from kinship_image_data_class import *

# from kinship_image_data_class import KinshipPairs
# from kinship_image_data_class import KinshipDataset

In [2]:
# Common
import pandas as pd
import random
import itertools
import numpy as np
from itertools import product
from tqdm import tqdm

# Files
from pathlib import Path
import os
from PIL import Image

# Machine Learning
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import models, transforms
from torchvision.models import VGG16_Weights, VGG19_Weights, ResNet50_Weights, ResNet152_Weights
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

# Face Net
from facenet_pytorch import InceptionResnetV1, MTCNN

# Other
from typing import List, Tuple, Optional



/home/dzega/.conda/envs/deepface_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Code

## Seed

In [3]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # For multi-GPU setups
    torch.backends.cudnn.deterministic = True  # Ensures deterministic behavior in cuDNN
    torch.backends.cudnn.benchmark = False  # Disables auto-tuning for deterministic behavior

SEED = 42
set_seed(seed=SEED)

# Note: This seed does not make the machin learning proccess and results deterministic.

## Data

In [4]:
# Get the current working directory (cwd)
current_dir = os.getcwd()

# Get the grandparent directory
grandparent_dir = os.path.abspath(os.path.join(current_dir, "..", ".."))

# Paths to data files
train_pairs_dir = os.path.join(grandparent_dir, "Data/families_in_the_wild/train/train-relationship-lists")
train_families_dir = os.path.join(grandparent_dir, "Data/families_in_the_wild/train/train-faces")

test_relationship_lists_dir = os.path.join(grandparent_dir, "Data/families_in_the_wild/test/test-relationship-lists")
test_faces_dir = os.path.join(grandparent_dir, "Data/families_in_the_wild/test/test-faces")
test_relationship_labels_dir = os.path.join(grandparent_dir, "Data/families_in_the_wild/test/test-relationship-labels")

In [5]:
balanced_classes = True
train_ratio = 0.8
validation_ratio = 0.1
test_ratio = 0.1

dataset = KinshipDataset(train_pairs_dir, train_families_dir,test_faces_dir,test_relationship_lists_dir,test_relationship_labels_dir, train_ratio=train_ratio, validation_ratio=validation_ratio, test_ratio=test_ratio, balanced_classes = balanced_classes)

train_pairs, validation_pairs, test_pairs, original_test_pairs = dataset.train_pairs, dataset.validation_pairs, dataset.test_pairs, dataset.original_test_pairs


train_dataset = KinshipPairs(train_pairs, train_families_dir)
validation_dataset = KinshipPairs(validation_pairs, train_families_dir)
test_dataset = KinshipPairs(test_pairs, train_families_dir)
original_test_dataset = KinshipPairs(original_test_pairs, test_faces_dir)

train_stats = dataset.get_statistics(train_pairs)
validation_stats = dataset.get_statistics(validation_pairs)
test_stats = dataset.get_statistics(test_pairs)


print(" ############ Created Datasets with the following characteristics: ############")
print(f"Balanced classes: {balanced_classes}")
print(f"Train ratio: {train_ratio}")
print(f"Validation ratio: {validation_ratio}")
print(f"Test ratio: {test_ratio}")
print("#### Families ####")
families = set(dataset.train_families_data_dict.keys())
print(f"Number of training families: {int(len(families)*(1-validation_ratio))}")
print(f"Number of validation families: {int(len(families)*validation_ratio)}")
print(f"Number of test families: {int(len(families)*test_ratio)}")
print("#### Set Pairs ####")
print("\nTrain Statistics:", train_stats)
print("\nValidation Statistics:", validation_stats)
print("\nTest Statistics:", test_stats)

 ############ Created Datasets with the following characteristics: ############
Balanced classes: True
Train ratio: 0.8
Validation ratio: 0.1
Test ratio: 0.1
#### Families ####
Number of training families: 565
Number of validation families: 62
Number of test families: 62
#### Set Pairs ####

Train Statistics: {'positive_pairs': 195386, 'negative_pairs': 195386, 'total_pairs': 390772}

Validation Statistics: {'positive_pairs': 61638, 'negative_pairs': 61638, 'total_pairs': 123276}

Test Statistics: {'positive_pairs': 20147, 'negative_pairs': 20147, 'total_pairs': 40294}


## Models

### Dence Classification Models (Face Pretrained Embedding Model Based)

In [9]:
def create_kinship_model(architecture):
    """
    Create a kinship model using pre-trained InceptionResnetV1 for face embeddings.
    
    :return: nn.Module, the kinship model
    """
    
    # Load pre-trained InceptionResnetV1 model from facenet-pytorch
    backbone = InceptionResnetV1(pretrained='vggface2').eval()  # Using the VGGFace2 dataset weights
    
    # Return the model instance
    return KinshipModel(backbone, architecture)

In [12]:
class KinshipModel(nn.Module):
    def __init__(self, encoder_backbone, architecture):
        super(KinshipModel, self).__init__()
        self.backbone = encoder_backbone  # Use the pre-trained face recognition model
        
        # Determine embedding dimension dynamically from the backbone
        dummy_input = torch.randn(1, 3, 160, 160)  # Example input size, matches Facenet's expected input
        self.embedding_dim = self.backbone(dummy_input).size(1)  # Get the embedding dimension from output size

        if architecture == "SEC":
            self.sequential_input_size = self.embedding_dim*2
        elif architecture == "SSC":
            self.sequential_input_size = self.embedding_dim + 2
        elif architecture == "SESC":
            self.sequential_input_size = self.embedding_dim*3 + 2
        else:
            raise Exception("Architecture must be astring from the following options: SEC, SSC, SESC")

        self.architecture = architecture

        ####################### Dense Classification Layers #######################
        # Binary classification head
        self.classifier = nn.Sequential(
            nn.Linear(self.sequential_input_size, 2048),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    ################################################################## Forward Function ##################################################################
    def forward(self, img1, img2):
        # Extract face embeddings for both images
        img1_embedding = self.backbone(img1)  # Shape: [batch_size, embedding_dim]
        img2_embedding = self.backbone(img2)  # Shape: [batch_size, embedding_dim]

        ############ Similarity Calculations ############
        # Calculate cosine similarity (scalar)
        cosine_sim = F.cosine_similarity(img1_embedding, img2_embedding, dim=1)
        # Calculate Euclidean distance (scalar)
        euclidean_dist = F.pairwise_distance(img1_embedding, img2_embedding, p=2)
        # Calculate squared difference (vector of shape [batch_size, embedding_dim])
        squared_diff = torch.pow(img1_embedding - img2_embedding, 2)

        # Classifier FFNN Input
        if self.architecture == "SEC":
            classifier_input = torch.cat((img1_embedding, img2_embedding), dim=1)
        elif self.architecture == "SSC":
            classifier_input = torch.cat((squared_diff, cosine_sim.unsqueeze(1), euclidean_dist.unsqueeze(1)), dim=1)
        elif self.architecture == "SESC":
            classifier_input = torch.cat((img1_embedding, img2_embedding, squared_diff, cosine_sim.unsqueeze(1), euclidean_dist.unsqueeze(1)), dim=1)

        # Feed the concatenated similarity metrics through the classifier
        out = self.classifier(classifier_input)
        return out

#### SEC (Siamese Embedding Concatenation)

In [7]:
class KinshipModel(nn.Module):
    def __init__(self, architecture):
        super(KinshipModel, self).__init__()
        self.backbone = architecture  # Use the pre-trained face recognition model
        
        # Determine embedding dimension dynamically from the backbone
        dummy_input = torch.randn(1, 3, 160, 160)  # Example input size, matches Facenet's expected input
        self.embedding_dim = self.backbone(dummy_input).size(1)  # Get the embedding dimension from output size

        ####################### Dense Classification Layers #######################
        # Binary classification head
        self.classifier = nn.Sequential(
            nn.Linear(self.embedding_dim*2, 2048),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    ################################################################## Forward Function ##################################################################
    def forward(self, img1, img2):
        # Extract face embeddings for both images
        img1_embedding = self.backbone(img1)  # Shape: [batch_size, embedding_dim]
        img2_embedding = self.backbone(img2)  # Shape: [batch_size, embedding_dim]

        # Concatenate all similarity metrics: [squared_diff, cosine_sim, euclidean_dist]
        concatenated_embeddings = torch.cat((img1_embedding, img2_embedding), dim=1)

        # Feed the concatenated similarity metrics through the classifier
        out = self.classifier(concatenated_embeddings)
        return out

#### SSC (Siamese Similarity Concatenation)

In [ ]:
class KinshipModel(nn.Module):
    def __init__(self, architecture):
        super(KinshipModel, self).__init__()
        self.backbone = architecture  # Use the pre-trained face recognition model
        
        # Determine embedding dimension dynamically from the backbone
        dummy_input = torch.randn(1, 3, 160, 160)  # Example input size, matches Facenet's expected input
        self.embedding_dim = self.backbone(dummy_input).size(1)  # Get the embedding dimension from output size

        ####################### Dense Classification Layers #######################
        # Binary classification head
        self.classifier = nn.Sequential(
            nn.Linear(self.embedding_dim + 2, 2048),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    ################################################################## Forward Function ##################################################################
    def forward(self, img1, img2):
        # Extract face embeddings for both images
        img1_embedding = self.backbone(img1)  # Shape: [batch_size, embedding_dim]
        img2_embedding = self.backbone(img2)  # Shape: [batch_size, embedding_dim]

        # Calculate cosine similarity (scalar)
        cosine_sim = F.cosine_similarity(img1_embedding, img2_embedding, dim=1)

        # Calculate Euclidean distance (scalar)
        euclidean_dist = F.pairwise_distance(img1_embedding, img2_embedding, p=2)

        # Calculate squared difference (vector of shape [batch_size, embedding_dim])
        squared_diff = torch.pow(img1_embedding - img2_embedding, 2)

        # Concatenate all similarity metrics: [squared_diff, cosine_sim, euclidean_dist]
        similarity = torch.cat((squared_diff, cosine_sim.unsqueeze(1), euclidean_dist.unsqueeze(1)), dim=1)

        # Feed the concatenated similarity metrics through the classifier
        out = self.classifier(similarity)
        return out

#### SESC (Siamese Embedding and Similarity Concatenation)

In [6]:
class KinshipModel(nn.Module):
    def __init__(self, architecture):
        super(KinshipModel, self).__init__()
        self.backbone = architecture  # Use the pre-trained face recognition model
        
        # Determine embedding dimension dynamically from the backbone
        dummy_input = torch.randn(1, 3, 160, 160)  # Example input size, matches Facenet's expected input
        self.embedding_dim = self.backbone(dummy_input).size(1)  # Get the embedding dimension from output size

        ####################### Dense Classification Layers #######################
        # Binary classification head
        self.classifier = nn.Sequential(
            nn.Linear(self.embedding_dim*3 + 2, 2048),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    ################################################################## Forward Function ##################################################################
    def forward(self, img1, img2):
        # Extract face embeddings for both images
        img1_embedding = self.backbone(img1)  # Shape: [batch_size, embedding_dim]
        img2_embedding = self.backbone(img2)  # Shape: [batch_size, embedding_dim]

        # Calculate cosine similarity (scalar)
        cosine_sim = F.cosine_similarity(img1_embedding, img2_embedding, dim=1)

        # Calculate Euclidean distance (scalar)
        euclidean_dist = F.pairwise_distance(img1_embedding, img2_embedding, p=2)

        # Calculate squared difference (vector of shape [batch_size, embedding_dim])
        squared_diff = torch.pow(img1_embedding - img2_embedding, 2)

        # Concatenate all similarity metrics: [squared_diff, cosine_sim, euclidean_dist]
        similarity = torch.cat((img1_embedding, img2_embedding, squared_diff, cosine_sim.unsqueeze(1), euclidean_dist.unsqueeze(1)), dim=1)

        # Feed the concatenated similarity metrics through the classifier
        out = self.classifier(similarity)
        return out

def create_kinship_model():
    """
    Create a kinship model using pre-trained InceptionResnetV1 for face embeddings.
    
    :return: nn.Module, the kinship model
    """
    
    # Load pre-trained InceptionResnetV1 model from facenet-pytorch
    backbone = InceptionResnetV1(pretrained='vggface2').eval()  # Using the VGGFace2 dataset weights
    
    # Return the model instance
    return KinshipModel(backbone)

#### Test embedding dimantions

In [8]:
import torch
from facenet_pytorch import InceptionResnetV1

# Load the pre-trained InceptionResnetV1 model
backbone = InceptionResnetV1(pretrained='vggface2').eval()

# Create a dummy input tensor (batch size 1, 3 color channels, 160x160 image)
dummy_input = torch.randn(1, 3, 160, 160)

# Pass the dummy input through the backbone model
embedding = backbone(dummy_input)

# Print the embedding dimension
print(f"Embedding shape: {embedding.shape}")  # Expected output: (1, embedding_dim)
print(f"Embedding dimension: {embedding.shape[1]}")  # The actual dimension value

Embedding shape: torch.Size([1, 512])
Embedding dimension: 512


## Training

### Functions

In [8]:
def evaluate_model(model, val_loader, device, threshold = 0.5):
    """
    This function is aimed at allowing to validate model performance by saving the true and predicted probabilities
    to then evaluate different thresholds and metrics
    :param model:
    :param val_loader:
    :param device:
    :return:
    """

    ######### Prep #########
    model_name = model.__class__.__name__
    model.eval()
    all_img1_paths, all_img2_paths = [],[]
    all_labels = []
    all_preds = []

    ######### Validation Feedforward #########
    with torch.no_grad():
        for img1, img2, labels, img1_paths, img2_paths in val_loader:
            img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)
            outputs = model(img1, img2).squeeze()

            all_img1_paths.extend(img1_paths)
            all_img2_paths.extend(img2_paths)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(outputs.cpu().numpy())

    ######### Save CSV #########
    results_df = pd.DataFrame()
    results_df['img1_path'] = all_img1_paths
    results_df['img2_path'] = all_img2_paths
    results_df['true'] = all_labels
    results_df['pred'] = all_preds
    # results_df.to_csv(f"{model_name}_Validation_Results.csv")

    ######### Evaluate Using Threshold #########
    binary_preds = [1 if p > threshold else 0 for p in all_preds]
    accuracy = accuracy_score(all_labels, binary_preds)
    roc_auc = roc_auc_score(all_labels, all_preds)  # Use raw probabilities for ROC AUC

    return accuracy, roc_auc, results_df

### Run Experiments

#### Inception Model

In [ ]:
#################################################################################### Init & Prep ####################################################################################
device = "cuda" if torch.cuda.is_available() else "cpu"

# Parameters
patience = 3
max_epochs = 20
criterion = nn.BCELoss()

print(f"Patience: {patience}")
print(f"Max Epochs: {max_epochs}")
print(f"Device Used: {device}")


#################################################################################### Data Loaders ####################################################################################

IS_TEST = False

if IS_TEST:
    # Subsample the datasets for quick testing
    sample_size = 1000  # Adjust this size as needed
    train_dataset_small = torch.utils.data.Subset(train_dataset, range(min(len(train_dataset), sample_size)))
    validation_dataset_small = torch.utils.data.Subset(validation_dataset,range(min(len(validation_dataset), sample_size)))
    test_dataset_small = torch.utils.data.Subset(test_dataset, range(min(len(test_dataset), sample_size)))

    # Create small data loaders
    train_loader = DataLoader(train_dataset_small, batch_size=128)  # Smaller batch size for quick tests
    validation_loader = DataLoader(validation_dataset_small, batch_size=128)
    test_loader = DataLoader(test_dataset_small, batch_size=128)

else:
    # Create data loaders
    print("\nCreating Data Loaders...")
    train_loader = DataLoader(train_dataset, batch_size=128)
    validation_loader = DataLoader(validation_dataset, batch_size=128)
    test_loader = DataLoader(test_dataset, batch_size=128)
    print("Finished")



#################################################################################### Run Experiments ####################################################################################

# Iterate over each architecture
for base_architecture in ['SEC','SSC','SESC']:

    print(f"\n\n\n\n############################################################# {base_architecture} #############################################################")
    
    # Model
    model = create_kinship_model(base_architecture)
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-5)

    # Initialization
    best_validation_auc = 0.0
    patience_counter = 0
    best_epoch = -1

    # Training using epochs
    for epoch in range(max_epochs):  # Set maximum epochs
        
        model.train()
        total_loss = 0
    
        ################################################# Training #################################################
        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}") # Initialize the progress bar for the epoch

        # Training Batches
        for batch_idx, (img1, img2, labels, img1_path, imag2_path) in progress_bar:
            img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(img1, img2).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
    
            
            progress_bar.set_postfix(loss=f"{total_loss / (batch_idx + 1):.4f}") # Update the progress bar with the current loss
            

        ################################################# Validation #################################################
        # Evaluate the model on the validation set
        model.eval()
        val_accuracy, val_auc, _ = evaluate_model(model, validation_loader, device)

        
        print(f"\n#### Epoch {epoch + 1} ####")
        print(f"Average Training Loss: {total_loss / len(train_loader):.4f}")
        print(f"Validation Accuracy: {val_accuracy:.4f}")
        print(f"Validation ROC AUC: {val_auc:.4f}")

        ################################################# Update Best Validation Score/Model #################################################
        # Check if validation performance has improved
        if val_auc > best_validation_auc:
            best_validation_auc = val_auc
            best_epoch = epoch
            patience_counter = 0
            # Save the best model based on validation performance
            Path("best models").mkdir(parents=True, exist_ok=True)
            torch.save(model.state_dict(), f"best models/best_kinship_{base_architecture}_model.pth")
            print("Validation AUC improved. Best model saved.")
        else:
            patience_counter += 1
            print(f"No improvement. Patience Counter: {patience_counter}/{patience}")
    
        # Early stopping condition
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break
    
        print("\n\n")
    
    print(f"\n\nTraining Complete.")
    
    
    ################################################# Test #################################################
    # Test the best model on the test set
    print("\nTesting on the Test Set...")
    model.load_state_dict(torch.load(f"best models/best_kinship_{base_architecture}_model.pth", weights_only=True))
    model.eval()
    
    results = []
    with torch.no_grad():
        for img1, img2, labels, img1_path, img2_path in tqdm(test_loader, desc="Testing"):
            img1, img2 = img1.to(device), img2.to(device)
            raw_outputs = model(img1, img2).squeeze()  # Raw predictions
            results.extend([
                {"label": label.item(), "predicted": raw_output.item(),
                 "image 1 full path": img1_path[i], "image 2 full path": img2_path[i]}
                for i, (label, raw_output) in enumerate(zip(labels, raw_outputs))
            ])

    # Calculate AUC for the test set
    y_true = [result["label"] for result in results]  # True labels
    y_pred = [result["predicted"] for result in results]  # Predicted probabilities
    test_auc = roc_auc_score(y_true, y_pred)
    print(f"Test ROC AUC: {test_auc:.4f}")

    ################################################# Saving Results #################################################

    # Create directories
    base_results_dir = Path("best model results")
    metrics_dir = base_results_dir / "metric results"
    csv_dir = base_results_dir / "test csv results"
    metrics_dir.mkdir(parents=True, exist_ok=True)
    csv_dir.mkdir(parents=True, exist_ok=True)
    
    # Save metrics to a text file
    metrics_file = metrics_dir / f"{base_architecture}_metrics.txt"
    with open(metrics_file, "w") as f:
        f.write(f"Architecture: {base_architecture}\n")
        f.write(f"Best Validation AUC: {best_validation_auc:.4f}\n")
        f.write(f"Best Epoch: {best_epoch + 1}\n")  # Epochs are 0-indexed in code
        f.write(f"Test AUC: {test_auc:.4f}\n")
    

    # Save the results as a CSV
    results_df = pd.DataFrame(results)
    test_csv_path = csv_dir / f"{base_architecture}_Test_Results.csv"
    results_df.to_csv(test_csv_path, index=False)
    print(f"Test results saved to {test_csv_path}")
    print(f"Metrics saved to {metrics_file}")

Patience: 3
Max Epochs: 20
Device Used: cuda

Creating Data Loaders...
Finished




############################################################# SEC #############################################################


Epoch 1: 100%|██████████| 3053/3053 [33:17<00:00,  1.53it/s, loss=0.3636] 



#### Epoch 1 ####
Average Training Loss: 0.3636
Validation Accuracy: 0.6664
Validation ROC AUC: 0.7410
Validation AUC improved. Best model saved.





Epoch 2: 100%|██████████| 3053/3053 [33:03<00:00,  1.54it/s, loss=0.1690] 



#### Epoch 2 ####
Average Training Loss: 0.1690
Validation Accuracy: 0.6985
Validation ROC AUC: 0.7931
Validation AUC improved. Best model saved.





...

Epoch 15: 100%|██████████| 3053/3053 [30:09<00:00,  1.69it/s, loss=0.0072]



#### Epoch 15 ####
Average Training Loss: 0.0072
Validation Accuracy: 0.7046
Validation ROC AUC: 0.8411
No improvement. Patience Counter: 1/3





Epoch 16: 100%|██████████| 3053/3053 [30:45<00:00,  1.65it/s, loss=0.0065] 



#### Epoch 16 ####
Average Training Loss: 0.0065
Validation Accuracy: 0.6933
Validation ROC AUC: 0.8412
No improvement. Patience Counter: 2/3





Epoch 17:  52%|█████▏    | 1581/3053 [15:30<14:24,  1.70it/s, loss=0.0063]

#### Old Method

In [15]:
#################################################################################### Init & Prep ####################################################################################
device = "cuda" if torch.cuda.is_available() else "cpu"

# Parameters
patience = 3
max_epochs = 20
criterion = nn.BCELoss()

print(f"Patience: {patience}")
print(f"Max Epochs: {max_epochs}")
print(f"Device Used: {device}")


#################################################################################### Data Loaders ####################################################################################

IS_TEST = True

if IS_TEST:
    # Subsample the datasets for quick testing
    sample_size = 1000  # Adjust this size as needed
    train_dataset_small = torch.utils.data.Subset(train_dataset, range(min(len(train_dataset), sample_size)))
    validation_dataset_small = torch.utils.data.Subset(validation_dataset,range(min(len(validation_dataset), sample_size)))
    test_dataset_small = torch.utils.data.Subset(test_dataset, range(min(len(test_dataset), sample_size)))

    # Create small data loaders
    train_loader = DataLoader(train_dataset_small, batch_size=64)  # Smaller batch size for quick tests
    validation_loader = DataLoader(validation_dataset_small, batch_size=64)
    test_loader = DataLoader(test_dataset_small, batch_size=64)

else:
    # Create data loaders
    print("\nCreating Data Loaders...")
    train_loader = DataLoader(train_dataset, batch_size=128)
    validation_loader = DataLoader(validation_dataset, batch_size=128)
    test_loader = DataLoader(test_dataset, batch_size=128)
    print("Finished")



#################################################################################### Run Experiments ####################################################################################

# Iterate over each architecture
for base_architecture in ["vggface2"]: # "vgg16","vgg19","resnet50","resnet152"

    print(f"\n\n\n\n############################################################# {base_architecture} #############################################################")
    
    # Model
    model = create_kinship_model(base_architecture, embedding_dim=256)
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    # Initialization
    best_validation_auc = 0.0
    patience_counter = 0
    best_epoch = -1

    # Training using epochs
    for epoch in range(max_epochs):  # Set maximum epochs
        
        model.train()
        total_loss = 0
    
        ################################################# Training #################################################
        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}") # Initialize the progress bar for the epoch

        # Training Batches
        for batch_idx, (img1, img2, labels, img1_path, imag2_path) in progress_bar:
            img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(img1, img2).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
    
            
            progress_bar.set_postfix(loss=f"{total_loss / (batch_idx + 1):.4f}") # Update the progress bar with the current loss
            

        ################################################# Validation #################################################
        # Evaluate the model on the validation set
        model.eval()
        val_accuracy, val_auc, _ = evaluate_model(model, validation_loader, device)

        
        print(f"\n#### Epoch {epoch + 1} ####")
        print(f"Average Training Loss: {total_loss / len(train_loader):.4f}")
        print(f"Validation Accuracy: {val_accuracy:.4f}")
        print(f"Validation ROC AUC: {val_auc:.4f}")

        ################################################# Update Best Validation Score/Model #################################################
        # Check if validation performance has improved
        if val_auc > best_validation_auc:
            best_validation_auc = val_auc
            best_epoch = epoch
            patience_counter = 0
            # Save the best model based on validation performance
            Path("best models").mkdir(parents=True, exist_ok=True)
            torch.save(model.state_dict(), f"best models/best_kinship_{base_architecture}_model.pth")
            print("Validation AUC improved. Best model saved.")
        else:
            patience_counter += 1
            print(f"No improvement. Patience Counter: {patience_counter}/{patience}")
    
        # Early stopping condition
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break
    
        print("\n\n")
    
    print(f"\n\nTraining Complete.")
    
    
    ################################################# Test #################################################
    # Test the best model on the test set
    print("\nTesting on the Test Set...")
    model.load_state_dict(torch.load(f"best models/best_kinship_{base_architecture}_model.pth", weights_only=True))
    model.eval()
    
    results = []
    with torch.no_grad():
        for img1, img2, labels, img1_path, img2_path in tqdm(test_loader, desc="Testing"):
            img1, img2 = img1.to(device), img2.to(device)
            raw_outputs = model(img1, img2).squeeze()  # Raw predictions
            results.extend([
                {"label": label.item(), "predicted": raw_output.item(),
                 "image 1 full path": img1_path[i], "image 2 full path": img2_path[i]}
                for i, (label, raw_output) in enumerate(zip(labels, raw_outputs))
            ])

    # Calculate AUC for the test set
    y_true = [result["label"] for result in results]  # True labels
    y_pred = [result["predicted"] for result in results]  # Predicted probabilities
    test_auc = roc_auc_score(y_true, y_pred)
    print(f"Test ROC AUC: {test_auc:.4f}")

    ################################################# Saving Results #################################################

    # Create directories
    base_results_dir = Path("best model results")
    metrics_dir = base_results_dir / "metric results"
    csv_dir = base_results_dir / "test csv results"
    metrics_dir.mkdir(parents=True, exist_ok=True)
    csv_dir.mkdir(parents=True, exist_ok=True)
    
    # Save metrics to a text file
    metrics_file = metrics_dir / f"{base_architecture}_metrics.txt"
    with open(metrics_file, "w") as f:
        f.write(f"Architecture: {base_architecture}\n")
        f.write(f"Best Validation AUC: {best_validation_auc:.4f}\n")
        f.write(f"Best Epoch: {best_epoch + 1}\n")  # Epochs are 0-indexed in code
        f.write(f"Test AUC: {test_auc:.4f}\n")
    

    # Save the results as a CSV
    results_df = pd.DataFrame(results)
    test_csv_path = csv_dir / f"{base_architecture}_Test_Results.csv"
    results_df.to_csv(test_csv_path, index=False)
    print(f"Test results saved to {test_csv_path}")
    print(f"Metrics saved to {metrics_file}")

Patience: 3
Max Epochs: 20
Device Used: cuda




############################################################# vggface2 #############################################################


  0%|          | 0.00/107M [00:00<?, ?B/s]

Epoch 1:   0%|          | 0/16 [00:00<?, ?it/s]


AttributeError: 'KinshipPairs' object has no attribute 'transform'

# Other